 # 🆕 Simple Trading Strategy #
 ***
  **Author:** Hamdan Sulaiman <br>
  **Created on:** 19th May 2021 <br>
  **Last Updated:** 19th May 2021 <br>
  **GitHub:**  https://github.com/msulaim/ib_insync-Tutorial/tree/simplestrategy/Simple_Strategy
 ***


**Key :**<br>
✅ *Compeleted*<br>

❌ *Incomplete*<br>

❗   *Problem*<br>
 
⏳  *In Progress*<br>


**Changes :**

| Date | Changes Made |
| :---: | :---: |
|05/19 | *simple_trading_strategy.ipynb* created |
|      | *Documented Untested Approach*
|      | *Added Problem related to Step 4*
|      | *Coded Untested Approach*            


***


**To Do :**

| To Do |
| :---: |
| test code during trading hours |
| code options contract and confirm what to do about missing data such as strike etc|



**Overview :**

- We want to now use our calculations of how much market value certain stocks and options have today vs what they will have at expiry

***
**Goals:**

- We will monitor a list of *10 potential stock tickers:*<br><br>
    
    - At any given time we will not have more than *10% of total combined Equity + Option value at expiry* in any one stock.
    
    - If the concentration in any one stock is *over 10%* we will recommend a *SELL*.
    
    - If the concentration is *below 7%* we will recommend a *BUY*. 
    
    - In both cases we will recommend the amount of *SELL or BUY* needed to get the concentration to *10%*.


- The list of 10 stocks can be predefined in the code, and hard coded for now :
    - *FB, AAPL, SQ, SHOP, TSLA, VIAC, TDOC, PTON, DAL, SPXL*

***



**Untested Approach:**

- Step 1 : Hardcode list of *Ticker Symbols*✅


- Step 2: 
    - Create a `Stock` using each `Ticker`symbol. An object of type `Stock` has the following parameters:✅

        - *symbol (str)* – Symbol name.

        - *exchange (str)* – Destination exchange.
    
        - *currency (str)* – Underlying currency.
    
    - Create an `Option` using each of `Ticker` symbol. An object of type `Option` has the following parameters: ❗ ❌
    
        - *symbol (str) – Symbol name.*
        
        - *strike (float) – The option’s strike price.*

        - *right (str) – Put or call option. Valid values are ‘P’, ‘PUT’, ‘C’ or ‘CALL’.*

        - *exchange (str) – Destination exchange.*

        - *multiplier (str) – The contract multiplier.*

        - *currency (str) – Underlying currency.*

        - *lastTradeDateOrContractMonth (str)*

- Step 3 : Qualify the contracts using ` ib.qualifyContracts(contract)`✅


- Step 4: Request ticker for each contract using `ib.reqTickers(contract) ` ⏳


- Step 5: Extract *Total Market Value* from the requested `Ticker` ⏳( ✅ for Stocks ) ( ❌ for Options)
> Explore the attributes of the `Ticker` as well


- Step 6 : Create an empty DataFrame with the following columns: Symbol, Exchange, Underlying Equity Market Value , Total Market Value Post Expiry, Percentage Market Value :✅<br><br> 
    - `% Market Value` =  Stock Value / Total combined Equity + Option value * 100 
    
- Step 7: Process the created DataFrame, applying conditional statements that fulfill the following:✅<br><br>
    
    - If the concentration in any one stock is *over 10%* we will recommend a *SELL*.
    
    - If the concentration is *below 7%* we will recommend a *BUY*. 


- Step 8: Add the recomendations to the DataFrame✅
***



**Problems :**

| Step | Step Description | Problem | Reason | Limitations Created | What we can do for the time being|
| :---: | :---: | :---: | :---: | :---: | :---: |
| Step 2 |Create an Option using each of Ticker symbol. An object of type Option has the following parameters: symbol, strike, right, expiration date | Unsure / Unable to create Contract for Option  |To create a contract for an Option, we also need the strike, right, expiration date, all of which are missing, before we extracted the contracts for Options from our Portfolio, these were already created, but this time we are hardcoding the symbols and creating their contracts from scratch | We cannot obtain post expiry market value without the Greek bids of the Option | We can continue working on Stocks only, whislt setting the Post Expiration value to 0 for options |


***


In [1]:
import numpy as np
import pandas as pd

#Import ib_insync library
from ib_insync import *

#Only used in interactive environments such as Jupyter Notebooks
util.startLoop()

#Instantiate IB class and use .connect() method on it, if TWS is not running ConnectionRefusedError will be raised
ib = IB()
try:
    print(ib.connect(clientId=0))
except:
    pass

<IB connected to 127.0.0.1:7497 clientId=0>


In [2]:
#List of Ticker Symbols
symbols = ['FB', 'AAPL', 'SQ', 'SHOP', 'TSLA', 'VIAC', 'TDOC', 'PTON', 'DAL', 'SPXL']

#Create object of type for each ticker symbol
##Only creating contracts for stocks not options
stock_contracts = [Stock(symbol=symbol, exchange='SMART', currency='USD') for symbol in symbols]

#Qualify the contracts
ib.qualifyContracts(*stock_contracts)

#Request Ticker for each qualified_contracts

## Requesting only for stock tickers
tickers = [ib.reqTickers(contract) for contract in stock_contracts]

In [3]:
#For Stocks

#Obtain the last price for the stock
stock_market_prices_last = [ticker[0].last for ticker in tickers]

#Obtain exchanges
exchanges = [ticker[0].dict()['contract'].primaryExchange for ticker in tickers]

#Combined Equity Value
combined_stocks_value = sum(stock_market_prices_last)


In [4]:
# For Options

#Obtain options post expiry market value
## Setting to 0 as if now
option_post_expiry_values = [0] * 10

#Combined Options Post Expiry Value
combined_options_post_expiry_value = sum(option_post_expiry_values)



In [5]:

#Create Dataframe
data = {'Symbol' : symbols, 'Exchange' : exchanges, 'Underlying Equity Market Value' : stock_market_prices_last, 'Total Market Value Post Expiry': option_post_expiry_values}
df_strategy = pd.DataFrame(data=data)


#Add colunm for % Market Value
df_strategy['% Market Value'] = (df_strategy['Underlying Equity Market Value'] / combined_stocks_value + combined_options_post_expiry_value) * 100

#Add empty column for Buy or Sell
df_strategy['Buy or Sell'] = np.nan


In [6]:
#Apply conditions, check for percentage values, if more than 10% Sell, if less than 7% Buy
df_strategy.loc[df_strategy['% Market Value'] > 10, ['Buy or Sell']] = 'Sell'
df_strategy.loc[df_strategy['% Market Value'] < 7, ['Buy or Sell']] = 'Buy'



In [7]:
df_strategy = df_strategy.set_index('Symbol')
display(df_strategy)

,Exchange,Underlying Equity Market Value,Total Market Value Post Expiry,% Market Value,Buy or Sell
Symbol,,,,,
FB,NASDAQ,311.48,0,11.259928,Sell
AAPL,NASDAQ,124.00,0,4.482570,Buy
SQ,NYSE,198.21,0,7.165244,NaN
SHOP,NYSE,1163.71,0,42.067839,Sell
TSLA,NASDAQ,560.57,0,20.264472,Sell
VIAC,NASDAQ,39.62,0,1.432254,Buy
TDOC,NYSE,135.22,0,4.888171,Buy
PTON,NASDAQ,95.07,0,3.436758,Buy
DAL,NYSE,45.96,0,1.661443,Buy


In [8]:
ib.disconnect()